Написать приложение и функцию, которые собирают основные новости с сайта на выбор lenta.ru, mail.ru . Для парсинга использовать XPath
Структура данных должна содержать:
* название источника
* наименование новости
* ссылку на новость
* дата публикации

минимум один сайт максимум все два

In [1]:
from pprint import pprint
from lxml import html
import requests
import re
from datetime import datetime

In [2]:
url_lenta='https://lenta.ru'
url_mail='https://news.mail.ru/'

headers={
    'User-Agent':	'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    
}

In [3]:
def news_from (url):
  response = requests.get(url, headers=headers)
  dom=html.fromstring(response.text)

  return dom

In [4]:
def get_news_lenta(dom):
  all_news_lenta=dom.xpath('//a[contains(@class, "card-")]')
  result_list=[]
  
  for news in all_news_lenta:
    time=news.xpath('.//time/text()')
    text=news.xpath('.//span[contains(@class, "card-")]/text()')
    source=news.xpath('./@href')

    links=[]
    source_name=[]    

    for link in source:

      link=str(link)
      removed_chars = "[]''"
      chars = '[%s]+' % re.escape(removed_chars)
      link=re.sub(chars, '', link)  
      
      if "http" in link:
        links.append(link)
        source_name.append(link[8:].split("/")[0])

      else:
        links.append(url_lenta+str(link)[2:])
        source_name.append("lenta.ru")

    result = {
            "text": text,
            "link": links,
            "time": time,
            "source": source_name
              }
    result_list.append(result)
  return result_list

In [5]:
def get_news_mail(dom):  
  all_news_mail=dom.xpath('//span[contains(@class, "item")]')
  result_list=[]

  for news in all_news_mail:
    source='mail.ru'
    time=str(datetime.today()).split(' ')[0]
    list_text=news.xpath('./a/text()')
    list_link=news.xpath('./a/@href')

    if len(list_text)==0:
      list_text=news.xpath('.//span[contains(@class, "__title")]/text()')
      
      if len(list_text)==0:
        list_text=news.xpath('../span[@class="newsitem__title-inner"]/text()')
        list_link=news.xpath('../span[@class="newsitem__title-inner"]/../../a/@href')

        if len(list_text)==0:
          list_text=news.xpath('../../../..//span[contains(@class, "link__text")]/div/div/text()')
          link=news.xpath('../../../..//span[contains(@class, "link__text")]/../@href')
          time=news.xpath('../../../..//span[contains(@class, "link__text")]/../..//a/div/span/span/div/time/text()')
          source=news.xpath('../../../..//span[contains(@class, "link__text")]/../..//a/div/span/span/div/text()')

          for el in link:
            list_link=(url_mail+str(el))
            
          
    if len(list_text)>0 and len(list_link)>0:    
      result={
          "text":list_text,
          "link":list_link,
          "source": source,
          "time": time
          }
      
      result_list.append(result)
  
  return result_list

In [6]:
def get_news_result(dom, source):

  if source==url_mail:
    result = get_news_mail(dom)
  elif source==url_lenta:
    result=get_news_lenta(dom)
  else:
    result = 'что-то пошло не так'

  return result

In [7]:
dom=news_from(url_mail)
get_news_result(dom, url_mail)

[{'text': ['Российские войска продолжили наступление на\xa0донецком направлении'],
  'link': ['https://news.mail.ru/politics/54542758/'],
  'source': 'mail.ru',
  'time': '2023-01-04'},
 {'text': ['Арестович осудил военных, выложивших буквы из\xa0убитых россиян'],
  'link': ['https://news.mail.ru/politics/54545493/'],
  'source': 'mail.ru',
  'time': '2023-01-04'},
 {'text': ['Поляку грозит до\xa0пяти лет тюрьмы за\xa0одобрение СВО на\xa0Украине'],
  'link': ['https://news.mail.ru/incident/54544282/'],
  'source': 'mail.ru',
  'time': '2023-01-04'},
 {'text': ['В городе Байконур пресечена деятельность ОПГ'],
  'link': ['https://news.mail.ru/incident/54541430/'],
  'source': 'mail.ru',
  'time': '2023-01-04'},
 {'text': ['Рогозин отправил ранивший его осколок снаряда послу Франции'],
  'link': ['https://news.mail.ru/politics/54541514/'],
  'source': 'mail.ru',
  'time': '2023-01-04'},
 {'text': ['В городе Байконур пресечена деятельность ОПГ'],
  'link': ['https://news.mail.ru/incident/5